In [68]:
from __future__ import absolute_import
from __future__ import print_function

import tensorflow as tf
import numpy as np

import os
import codecs
import nltk.data
import collections
from glob import glob
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords as nltk_stopwords

punctuation_remover = RegexpTokenizer(r'\w+')
stopwords = nltk_stopwords.words('english')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sess = tf.InteractiveSession()

In [3]:
with codecs.open(filename, 'r', 'utf-8') as f:
    sentences = tokenizer.tokenize(f.read())

sentences[1]
nameword2idx.keys() 

NameError: name 'nameword2idx' is not defined

In [76]:
with open(os.path.join('./data', 'name.txt')) as f:
    name_lists = f.readlines()

names = [name.strip().split('\t') for name in name_lists]

#name_counter = collections.Counter([word for name_set in names for name in name_set for word in name.split()])
#print(name_counter.most_common(100))

name2idx = {}
nameword2idx = {}

for idx, name_set in enumerate(names):
    for name in name_set:
        word_in_name = name.split()
        name_without_punctuation = " ".join(punctuation_remover.tokenize(name))

        for name in [name, name_without_punctuation]:
            try:
                if idx not in name_dict[name]:
                    name2idx[name].append(idx)
            except:
                name2idx[name] = []
                name2idx[name].append(idx)
        
        for name in word_in_name:
            try:
                if idx not in name_dict[name]:
                    nameword2idx[name].append(idx)
            except:
                nameword2idx[name] = []
                nameword2idx[name].append(idx)

idx2name = dict(zip(name2idx.values(), name2idx.keys()))

TypeError: unhashable type: 'list'

In [77]:
def stopword_filter(text):
    return " ".join([word for word in text.split() if word not in stopwords])

In [131]:
name_lists[0]

'Hogwarts\n'

In [138]:
vocab_size = 50000

def read_name_data(data_dir):
    with open(os.path.join(data_dir, 'name.txt')) as f:
        name_lists = f.readlines()
        
    names = [name.lower().strip().split('\t') for name in name_lists]

    #name_counter = collections.Counter([word for name_set in names for name in name_set for word in name.split()])
    #print(name_counter.most_common(100))

    name2idx = {}
    nameword2idx = {}

    for idx, name_set in enumerate(names):
        for name in name_set:
            name2idx[name] = idx
            word_in_name = name.split()
            name_without_punctuation = " ".join(punctuation_remover.tokenize(name))

            for name in [name, name_without_punctuation] + word_in_name:
                try:
                    if idx not in name_dict[name]:
                        name2idx[name].append(idx)
                except:
                    nameword2idx[name] = []
                    nameword2idx[name].append(idx)

    idx2name = dict(zip(name2idx.values(), name2idx.keys()))
    
    return names, name2idx, idx2name, nameword2idx

def read_data(data_dir):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    
    text = ""
    for filename in glob(os.path.join(data_dir, "*.txt")):
        if 'name.txt' not in filename:
            continue
        with open(filename) as f:
            text += f.read()
    tokenizer.tokenize(text)
    return text.split()

def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocab_size - 1))

    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)

    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count = unk_count + 1
        data.append(index)

    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))

    return dictionary, reverse_dictionary, data, count

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    for i in range(batch_size // num_skips):
        target = skip_window    # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    return batch, labels

In [139]:
def generate_batch(batch_size, num_skips, skip_window):
    pass

In [140]:
words = read_data('./data/')
print('Data size :', len(words))

names, name2idx, idx2name, nameword2idx = read_name_data("./data/")
print('# of names :', len(names))

dictionary, reverse_dictionary, data, count = build_dataset(words)
print('Most common words (+UNK) :', count[:5])

data_index = 0

Data size : 396
# of names : 189
Most common words (+UNK) : [['UNK', 0], ('Weasley', 9), ('Potter', 6), ('Longbottom', 4), ('Dursley', 4)]


In [143]:
sentences = []

for filename in glob(os.path.join('./data', "*.txt")):
    if 'name.txt' in filename:
        continue
    with open(filename, 'r') as f:
        nltk_splited_sentences = tokenizer.tokenize(stopword_filter(f.read().decode('utf-8').lower()))
        current_sentences = [sentence for sentence in nltk_splited_sentences
                             if any(word in nameword2idx.keys() for word in sentence.split())]
        sentences.extend(current_sentences[1:])
        
        print(" [*] %s finished: %d / %d" % (filename, len(current_sentences), len(nltk_splited_sentences)))

print(" [*] Total sentences : %d" % (len(sentences)))

 [*] ./data/1.txt finished: 2110 / 4861
 [*] ./data/6.txt finished: 3984 / 8235
 [*] ./data/2.txt finished: 2527 / 5148
 [*] ./data/5.txt finished: 6316 / 12451
 [*] ./data/7.txt finished: 4510 / 10492
 [*] ./data/4.txt finished: 5034 / 10519
 [*] ./data/3.txt finished: 3232 / 6614
 [*] Total sentences : 27706


In [130]:
print(stopword_filter(open('./data/1.txt').read().decode('utf-8').lower())[:1000])

harry potter sorcerer’s stone j. k. rowling say you’ve spent first 10 years life sleeping stairs family loathes you. then, absurd, magical twist fate find surrounded wizards, caged snowy owl, phoenix-feather wand, jellybeans come every flavor, including strawberry, curry, grass, sardine. that, discover wizard yourself! exactly happens young harry potter j. k. rowling’s enchanting, funny debut novel, harry potter sorcerer’s stone. nonmagic human world—the world “muggles”—harry nobody, treated like dirt aunt uncle begrudgingly inherited parents killed evil voldemort. world wizards, small, skinny harry famous survivor wizard tried kill him. left lightning-bolt scar forehead, curiously refined sensibilities, host mysterious powers remind he’s quite, yes, altogether different aunt, uncle, spoiled, piglike cousin dudley. mysterious letter, delivered friendly giant hagrid, wrenches harry dreary, muggle-ridden existence: “we pleased inform accepted hogwarts school witchcraft wizardry.” course,

In [144]:
for s in sentences[:5]:
    print(s)

exactly happens young harry potter j. k. rowling’s enchanting, funny debut novel, harry potter sorcerer’s stone.
world wizards, small, skinny harry famous survivor wizard tried kill him.
mysterious letter, delivered friendly giant hagrid, wrenches harry dreary, muggle-ridden existence: “we pleased inform accepted hogwarts school witchcraft wizardry.” course, uncle vernon yells unpleasantly, “i paying crackpot old fool teach magic tricks!” soon enough, however, harry finds hogwarts owl hedwig… that’s real adventure—humorous, haunting, suspenseful—begins.
harry potter sorcerer’s stone, first published england harry potter philosopher’s stone, continues win major awards england.
1. boy lived mr. mrs. dursley, number four, privet drive, proud say perfectly normal, thank much.


In [92]:
x = [sentence for sentence in tokenizer.tokenize(codecs.open('./data/1.txt', 'r', 'utf-8').read())]

In [ ]:
print(np.ones([3,2,1]))
print(tf.reduce_sum(np.ones([3,2,1]),0).eval())

In [ ]:
y = tf.placeholder(tf.int32, [None])

b = tf.Variable(np.array([[0], [1], [2], [3], [4], [5]]), name="b")
embed = tf.nn.embedding_lookup(b, y)
reshape = tf.reshape(embed, [3])

init = tf.initialize_all_variables()
sess.run(init)

print(embed.eval({y:[1,4,1]}))
print(reshape.eval({y:[1,4,1]}))
print(tf.add(tf.zeros([vocab_size]).eval(),1).eval())

In [ ]:
def model_factory(vocab_size):
    embed_size = 200
    batch_size = 16
    neg_sample_size = 6

    x = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    neg_y = tf.placeholder(tf.int32, [neg_sample_size])

    init_width = 0.5 / embed_size

    embed = tf.Variable(tf.random_uniform([vocab_size, embed_size], -init_width, init_width))
    w = tf.Variable(tf.zeros([vocab_size, embed_size]))
    b = tf.Variable(tf.zeros([vocab_size]))

    pos_embed = tf.nn.embedding_lookup(embed, x) # [batch_size x embed_size]
    pos_w = tf.nn.embedding_lookup(w, y)         # [batch_size x embed_size]
    pos_b = tf.nn.embedding_lookup(b, y)         # [batch_size x 1]

    pos_y_ = tf.add(tf.reduce_sum(tf.mul(pos_embed, pos_w), 1), pos_b) # [batch_size]

    # neg_embed = pos_embed
    neg_w = tf.nn.embedding_lookup(w, neg_y)     # [neg_sample_size x embed_size]
    neg_b = tf.nn.embedding_lookup(b, neg_y)     # [neg_sample_size]

    neg_y_ = tf.matmul(pos_embed, neg_w, transpose_b=True) + neg_b # [batch_size x neg_sample_size]

    pos_y = tf.ones_like(pos_y_)
    neg_y = tf.ones_like(neg_y_)

    pos_loss = tf.nn.sigmoid_cross_entropy_with_logits(pos_y_, tf.ones_like(pos_y_))
    neg_loss = tf.nn.sigmoid_cross_entropy_with_logits(neg_y_, tf.zeros_like(neg_y_))

    ##################
    # Optimizer
    ##################

    global_step = tf.Variable(0, name="global_step")

    total_word_processed = float(word_per_epoch * epochs_to_train)

    learning_rate = 0.01
    lr = learning_rate * tf.maximum(
        0.001,
        1.0 - tf.cast(total_word_processed, tf.float32) / words_to_train
    )

    loss = (tf.reduce_sum(pos_loss) + tf.reduce_sum(neg_loss))/batch_size
    train = tf.train.GradientDescentOptimizer(lr).minimize(loss,
                                                           global_step = global_step,
                                                           )

In [ ]:
flags = tf.app.flags

flags.DEFINE_string("data_dir", './data/', "Directory which contains data files")

FLAGS = flags.FLAGS

class Options(object):
    def __init__(self):
        self.data_dir = FLAGS.data_dir

def main():
    if not FLAGS.data_dir:
        print("--data_dir must be specified")
        sys.exit(1)

    opts = Options()
    read_data(opts.data_dir)
